In [1]:
import theano
import numpy as np
import time
import skimage.io

import sys
sys.path.insert(0, '/home/flipvanrijn/Workspace/Dedicon-Thesis/')

%load_ext autoreload
%autoreload 2

Couldn't import dot_parser, loading of dot files will not be possible.


Using gpu device 0: Quadro K2200


In [2]:
%%time
from networks.keras.vocab import Vocab

vocab = Vocab('/home/flipvanrijn/Workspace/Dedicon-Thesis/downloads/coco_train_dataset.json')
print 'Vocabulary loaded'

Vocabulary loaded
CPU times: user 2.84 s, sys: 184 ms, total: 3.03 s
Wall time: 3.03 s


In [27]:
%%time
from networks.keras.reversetime import ReverseTime
from keras.layers.recurrent import LSTM, SimpleRNN
from keras.layers.embeddings import Embedding
from keras.layers.core import TimeDistributedDense, Activation, Dense
from keras.models import Graph

n_symbols = vocab.token_count
rnn_dim = 1024
embedding_dim = 1000
dense_dim = 400
output_dim = 1

model = Graph()
model.add_input(name='sent', ndim=2, dtype=int)
model.add_node(
    Embedding(input_dim=n_symbols + 1, output_dim=embedding_dim, mask_zero=True),
    name='sent_embedding',
    input='sent'
)
model.add_node(
    SimpleRNN(input_dim=embedding_dim, output_dim=rnn_dim),
    name='in_rnn',
    input='sent_embedding'
)
model.add_node(
    ReverseTime(),
    name='sent_embedding_reverse',
    input='sent_embedding',
)
model.add_node(
    SimpleRNN(input_dim=embedding_dim, output_dim=rnn_dim),
    name='in_rnn_reverse',
    input='sent_embedding_reverse',
)
model.add_node(
    ReverseTime(),
    name='in_rnn_reverse_reverse',
    input='in_rnn_reverse'
)
model.add_node(
    Dense(rnn_dim, 30),
    name='dense1',
    merge_mode='sum',
    inputs=['in_rnn_reverse_reverse', 'in_rnn']
)
model.add_node(
    Activation('sigmoid'),
    name='final_activation',
    input='dense1'
)
model.add_output(name='output', input='final_activation')
model.compile('rmsprop', {'output': 'mse'})

TypeError: 'NoneType' object has no attribute '__getitem__'

In [26]:
from networks.keras.batchgenerator import BatchGenerator
from IPython import embed

bg = BatchGenerator(vocab, batch_size=1024)

for Ys, Xs in bg.get_data():
    print '{}/{}'.format(bg.current_batch+1, bg.total_batches)
    model.fit(data={'sent': Xs, 'output': Xs}, nb_epoch=1)
    #preds = model.predict_on_batch({'sent': Xs})
    #embed()

1/405
Epoch 0
1024/1024 [==============================] - 0s - output: 9880118.6250     
2/405
Epoch 0
1024/1024 [==============================] - 0s - output: 10123462.3750     
3/405
Epoch 0
1024/1024 [==============================] - 0s - output: 10097853.3750     
4/405
Epoch 0
1024/1024 [==============================] - 0s - output: 9861897.6250     
5/405
Epoch 0
1024/1024 [==============================] - 0s - output: 10408571.3750     
6/405
Epoch 0
1024/1024 [==============================] - 0s - output: 10148129.3750     


KeyboardInterrupt: 

[array([[-0.04738388,  0.01228474, -0.0396749 , ...,  0.04110826,
         -0.02542996,  0.00981489],
        [-0.02379394, -0.02635051,  0.01085035, ...,  0.00790174,
         -0.00920718,  0.04686673],
        [-0.0281559 ,  0.01170281, -0.04226818, ...,  0.04586127,
          0.04269754, -0.03629366],
        ..., 
        [ 0.01507534, -0.00569347,  0.02556291, ...,  0.0025896 ,
         -0.00549766, -0.01732519],
        [ 0.03060252, -0.02776979, -0.02507271, ..., -0.03963024,
         -0.0312012 , -0.04170848],
        [ 0.01909568,  0.02954179,  0.0403088 , ...,  0.03661922,
         -0.00401173, -0.02918692]], dtype=float32),
 array([[-0.01555539,  0.04980698, -0.06409916, ..., -0.0488957 ,
         -0.05432528,  0.05117698],
        [-0.05831897,  0.00593725, -0.01372765, ...,  0.08075752,
          0.03816903, -0.02233946],
        [-0.06956814, -0.00604892, -0.01280623, ...,  0.02647288,
         -0.07809586,  0.0731915 ],
        ..., 
        [-0.04202582, -0.02126872,  0